In [9]:
from matplotlib import pyplot
%matplotlib inline

from netCDF4 import Dataset
import datetime
import sys
import numpy
from mpl_toolkits.basemap import Basemap
import math

In [10]:
datafolder = '/home/clare/GoogleDrive/PolarCOAWST/CICE/CICE-runs/'
figfolder = '/home/clare/GoogleDrive/PythonScripts/CICE_outputs/figures/'
fh = 'iceh.1998-'
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
#months = [3]
varnames = ['aice', 'sst', 'hi']
#varnames = ['aice']

# STILL NEED TO SET UP CAPABILITY TO LOOP THROUGH PROJECTIONS!!!
#map_projections = ['cyl',]

In [11]:
for im in months:
    mm = str(im)
    if len(mm) < 2 :
        imon = '0' + mm
    else:
        imon = mm
    
    fname = fh + imon + '.nc'
    
    ncfile = Dataset(datafolder + fname, 'r')
    # collect common vars
    Tlats = ncfile.variables['TLAT']
    Tlons = ncfile.variables['TLON']
    Ulats = ncfile.variables['ULAT']
    Ulons = ncfile.variables['ULON']
    itime = ncfile.variables['time']
    #tmask = numpy.array(ncfile.variables['tmask']) # ocean grid mask

    if itime.units[0:4] == 'days' and len(itime) == 1 :
        dd = int(itime.units[-11:-9])
        mm = int(itime.units[-14:-12])
        year = int(itime.units[-19:-15])
        time0 = datetime.date(year, mm, dd)
        time = time0 + datetime.timedelta(days = int(itime[0]))
    else:
        sys.exit("time variable needs processing!")
        
    
    for ivar in varnames:
        # for now assume 1 field
        
        pltvar = ncfile.variables[ivar]
        units = pltvar.units
        long_name = pltvar.long_name
        if pltvar.shape[0] != 1 :
            sys.exit("variable shape is unexpected! Add functionality for matrices")
        
        coords = getattr(pltvar, 'coordinates')
        if "TLON" in coords :
            lats = numpy.array(Tlats)
            lons = numpy.array(Tlons)
        elif "ULON" in coords :
            lats = Ulats
            lons = Ulons
        else:
            sys.exit("check coordinates of variable ", ivar)

        
        pltvar = numpy.array(pltvar[0,:,:])
        pltvar = numpy.ma.masked_greater(pltvar, 1e20)
        if ivar == 'aice' or 'hi' :
            pltvar = numpy.ma.masked_equal(pltvar, 0.)
        clevs = numpy.linspace(math.floor(pltvar.min()), 
                          math.ceil(pltvar.max()), 21)
        pltvar_SP = numpy.where(lats>-50., 1e30, pltvar)
        pltvar_SP = numpy.ma.masked_greater(pltvar_SP, 1e20)
        clevs_SP =  numpy.linspace(math.floor(pltvar_SP.min()), 
                          math.ceil(pltvar_SP.max()), 21)

        #-----------------------------------
        # plot all data on 
        #-----------------------------------
        
        fig = pyplot.figure(im)
        m = Basemap(llcrnrlon=0,llcrnrlat=-80,urcrnrlon=360,
                       urcrnrlat=80,projection='mill')
        m.drawcoastlines(linewidth = 0.25)
        m.fillcontinents(color = '0.9')
        m.drawparallels(numpy.arange(-80.,81.,20.),labels=[1,1,0,0])
        m.drawmeridians(numpy.arange(0.,360.,60.),labels=[0,0,0,1])
        #x, y = m(lons,lats)
        cmap = pyplot.get_cmap('Spectral')
        p = m.pcolor(lons, lats, pltvar, cmap = cmap, 
                              alpha = 1.0, latlon = True)#, vmin=0.0001, vmax=10000.)
        # NB save as eps ignores alpha!
        cb = m.colorbar(p, "right", size = "5%", pad = "15%")
        pyplot.title(ivar + ', ' + long_name + ' [' + units +']' )
        pyplot.savefig(figfolder + ivar + imon + 
                       '_mill.eps', format = 'eps', dpi = 1000)
        pyplot.close()
        
        #--------------------------------------
        # plot South pole data on splaea grid
        #      - also try spstere
        #--------------------------------------
        
        fig2 = pyplot.figure(im*100)
        m2 = Basemap(projection = 'splaea', boundinglat = -50.0, 
                     lon_0 = 180., round = True)
        m2.drawcoastlines(linewidth = 0.25)
        m2.fillcontinents(color = '0.9')
        m2.drawparallels(numpy.arange(-90., -20., 10.))
        m2.drawmeridians(numpy.arange(0., 360., 60.),labels=[1,1,1,1])
        x, y = m2(lons,lats) # get lat and lon arrays for contour plot
        c = m2.contourf(x, y, pltvar, cmap = cmap, levels = clevs_SP)

        pyplot.colorbar(orientation='horizontal', shrink=0.65)
        pyplot.title(ivar + ', ' + long_name + ' [' + units +']' )
        pyplot.savefig(figfolder + ivar + imon + 
                       '_SP.eps', format = 'eps', dpi = 1000)
        pyplot.close()
        
ncfile.close()